# Data Exploration Notebook (Reproducible)

This notebook reproduces the findings summarized in `data/raw/DATA_EXPLORATION_SUMMARY.md`. Run each section to regenerate the results.

## 1) Environment and Configuration

In [ ]:
import os, sys, json, re
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

# Add src to path
ROOT = Path.cwd().parents[0] if (Path.cwd().name == 'notebooks') else Path.cwd()
if (ROOT / 'src').exists():
    sys.path.insert(0, str(ROOT))
from src.config import NUM_PARTICLES, NUM_FEATURES, FEATURE_NAMES, DATASET_FILES

print('Environment ready')
print('NUM_PARTICLES:', NUM_PARTICLES)
print('NUM_FEATURES:', NUM_FEATURES)
print('FEATURE_NAMES:', FEATURE_NAMES)
print('DATASET_FILES configured:', len(DATASET_FILES))
print('numpy:', np.__version__)
print('pandas:', pd.__version__)
print('h5py:', h5py.__version__)


## 2) Dataset Availability

In [ ]:
data_dir = ROOT / 'data' / 'raw'
print('Looking in:', data_dir)
avail = []
for fn in DATASET_FILES:
    p = data_dir / fn
    print(('✅' if p.exists() else '❌'), fn, ('(' + str(round(p.stat().st_size/1024/1024,1)) + ' MB)') if p.exists() else '(missing)')
    if p.exists():
        avail.append(p)
print(f'Found {len(avail)} / {len(DATASET_FILES)} files')


## 3) HDF5 Structure (Representative File)

In [ ]:
rep = avail[0] if avail else None
print('Representative file:', rep.name if rep else 'None')
if rep:
    with h5py.File(rep, 'r') as f:
        keys = list(f.keys())
        print('Top-level keys:', keys)
        def show(name, obj):
            if isinstance(obj, h5py.Dataset):
                size_mb = obj.size * obj.dtype.itemsize / 1024/1024
                print(f' - {name}: shape={obj.shape}, dtype={obj.dtype}, ~{size_mb:.2f} MB')
        f.visititems(show)


## 4) Physics Parameter Parsing from Filenames

In [ ]:
pattern = re.compile(r'mZprime-(?P<mZprime>\d+)_mDark-(?P<mDark>\d+)_rinv-(?P<rinv>0\.\d+)_alpha-(?P<alpha>\w+)')
rows = []
for p in avail:
    fn = p.name
    if 'NominalSM' in fn:
        rows.append({'filename': fn, 'mZprime': None, 'mDark': None, 'rinv': None, 'alpha': 'SM'})
    else:
        m = pattern.search(fn)
        if m:
            d = m.groupdict()
            d['mZprime'] = int(d['mZprime'])
            d['mDark'] = int(d['mDark'])
            d['rinv'] = float(d['rinv'])
            d['filename'] = fn
            rows.append(d)
df = pd.DataFrame(rows)
display(df)
for col in ['mZprime','mDark','rinv','alpha']:
    if col in df.columns:
        print(col, sorted(df[col].dropna().unique().tolist()))
print('Counts: total files', len(df), '| signal', (df['alpha']!='SM').sum(), '| background', (df['alpha']=='SM').sum())


## 5) Single-File Analysis

In [ ]:
import numpy as np
if rep:
    with h5py.File(rep, 'r') as f:
        n = f['jet_pt'].shape[0] if 'jet_pt' in f else None
        dark = f['jet_is_dark'][:] if 'jet_is_dark' in f else None
        n_dark = int(dark.sum()) if dark is not None else None
        print('Total jets:', n)
        if n_dark is not None:
            print('Dark jets:', n_dark, f'({n_dark/n*100:.1f}%)')
        if 'particle_features' in f:
            pf = f['particle_features'][:]
            print('particle_features shape:', pf.shape)
            arr = pf.reshape(-1, NUM_PARTICLES, NUM_FEATURES)
            invalid = np.isclose(arr, -999).any(axis=2)
            valid_frac = 1.0 - invalid.mean()
            print('Estimated valid particle fraction (sample):', f'{valid_frac:.3f}')
        for k in ['jet_pt','jet_eta','jet_phi','jet_m','jet_e','jet_rinv']:
            if k in f:
                print(k, f[k].shape)


## 6) Feature Distributions Across Files (valid particles)

In [ ]:
def load_pf(p):
    with h5py.File(p, 'r') as f:
        pf = f['particle_features'][:]
        is_dark = f['jet_is_dark'][:] if 'jet_is_dark' in f else None
        return pf, is_dark

sig_pT = []; sig_eta = []; sig_phi = []
sm_pT = []; sm_eta = []; sm_phi = []
for p in avail:
    pf, is_dark = load_pf(p)
    arr = pf.reshape(-1, NUM_PARTICLES, NUM_FEATURES)
    invalid = np.isclose(arr, -999).any(axis=2)
    valid = ~invalid
    if 'NominalSM' in p.name:
        sm_pT.append(arr[:,:,0][valid])
        sm_eta.append(arr[:,:,1][valid])
        sm_phi.append(arr[:,:,2][valid])
    else:
        if is_dark is not None:
            vm = valid[is_dark]  # only dark jets
            arr_d = arr[is_dark]
            sig_pT.append(arr_d[:,:,0][vm])
            sig_eta.append(arr_d[:,:,1][vm])
            sig_phi.append(arr_d[:,:,2][vm])
sig = {'pT': np.concatenate(sig_pT), 'eta': np.concatenate(sig_eta), 'phi': np.concatenate(sig_phi)}
sm  = {'pT': np.concatenate(sm_pT),  'eta': np.concatenate(sm_eta),  'phi': np.concatenate(sm_phi)}

def describe(v):
    v = v[np.isfinite(v)]
    return dict(count=int(v.size), mean=float(v.mean()), std=float(v.std()), min=float(v.min()), max=float(v.max()))

for ftr in ['pT','eta','phi']:
    ds = describe(sig[ftr]); db = describe(sm[ftr])
    print(ftr, 'Signal', ds, '| SM', db)
# Optionally plot (density)
plt.figure(); plt.hist(sm['pT'], bins=100, density=True, alpha=0.6, label='SM'); plt.hist(sig['pT'], bins=100, density=True, alpha=0.6, label='Signal'); plt.legend(); plt.title('pT (valid particles)'); plt.show()


## 7) Jet-level Feature Distributions

In [ ]:
JET_KEYS = ['jet_pt','jet_eta','jet_phi','jet_m','jet_e']
sig = {k: [] for k in JET_KEYS}
sm = {k: [] for k in JET_KEYS}
for p in avail:
    with h5py.File(p, 'r') as f:
        is_sm = 'NominalSM' in p.name
        is_dark = f['jet_is_dark'][:] if 'jet_is_dark' in f else None
        for k in JET_KEYS:
            if k not in f: continue
            arr = f[k][:]
            if is_sm:
                sm[k].append(arr)
            else:
                if is_dark is None: continue
                sig[k].append(arr[is_dark])
sig = {k: np.concatenate(v) for k,v in sig.items()}
sm  = {k: np.concatenate(v) for k,v in sm.items()}

def d(v):
    v = v[np.isfinite(v)]
    return dict(count=int(v.size), mean=float(v.mean()), std=float(v.std()), min=float(v.min()), max=float(v.max()))

for k in JET_KEYS:
    print(k, 'Signal', d(sig[k]), '| SM', d(sm[k]))
# Example plot
plt.figure(); plt.hist(sm['jet_pt'], bins=100, density=True, alpha=0.6, label='SM'); plt.hist(sig['jet_pt'], bins=100, density=True, alpha=0.6, label='Signal'); plt.legend(); plt.title('jet_pt (jets)'); plt.show()


### Notes
- Histograms are plotted with `density=True` to compare shapes irrespective of different sample sizes.
- No training merges/splits are performed here; these are analysis-only aggregates.